# Super-structure scenario modelling in brightway

This notebook shows how to use the bw_scenarios module, using the example of a bike.

We start by importing an SDF file (.xslx or .csv).

## Project setup

In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc

In [2]:
PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"

In [3]:
bd.projects.set_current(PROJECT_NAME)

In [4]:
bd.databases

Databases dictionary with 2 object(s):
	bike_production_example
	ecoinvent-3.10-biosphere

In [5]:
xl_importer = bi.importers.ExcelImporter("bike_production_example.xlsx")

Extracted 1 worksheets in 0.02 seconds


In [6]:
# If there are unlinked exchanges because of biosphere name change,
# use the right biosphere name

In [7]:
for data_item in xl_importer.data:
    for exchange in data_item["exchanges"]:
        if exchange["type"] == "biosphere" and exchange["database"] == "biosphere3":
            print(f"Update bio db name for {exchange}")
            exchange["database"] = BIO_DB_NAME

Update bio db name for {'name': 'Carbon dioxide, fossil', 'amount': 26.6, 'database': 'biosphere3', 'unit': 'kilogram', 'categories': 'air', 'type': 'biosphere'}


In [8]:
xl_importer.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 14.34 seconds


In [9]:
xl_importer.statistics()

3 datasets
	8 exchanges
	Links to the following databases:
		bike_production_example (7 exchanges)
		ecoinvent-3.10-biosphere (1 exchanges)
	0 unlinked exchanges (0 types)
		


(3, 8, 0, 0)

In [10]:
if xl_importer.statistics(print_stats=False)[2] == 0:
    xl_importer.write_database()

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 621.35it/s]

Vacuuming database 
Created database: bike_production_example


In [34]:
from bw_scenarios.scenario import Scenario

In [ ]:
scenario_lca = Scenario

In [11]:
bike_db = bd.Database("bike_production_example")

In [12]:
for a in bike_db:
    print(f"{a} -> {a['code']}")

'natural gas production' (mega joule, NO, None) -> 6ba895871c694e9a898ade17ab6748e4
'production of bike' (unit, DK, None) -> 1d31f61bf8884a6fa5b3783270091f9d
'carbon fibre production' (kilogram, DE, None) -> 7bde0c388a1b401c95dccd0a3429bd0c


In [13]:
#make mock up for scenario.exchanges
exc_dict = {}
for act in bike_db:
    if act.technosphere():
        exc_dict[act.id] = []
    
    for exc in act.technosphere():
        # print("from_id:", exc.input.id, exc.input["name"], exc.amount, exc["type"])
        # print("To_id:", act.id, act["name"])
        
        # print("\n")
        exc_dict[act.id].append((exc.input.id, exc["type"], exc.amount))

    for exc in act.biosphere():
        # print("from_id:", exc.input.id, exc.input["name"], exc.amount, exc["type"])
        # print("To_id:", act.id, act["name"])
        
        # print("\n")
        exc_dict[act.id].append((exc.input.id, exc["type"], exc.amount))

print(exc_dict)

{4365: [(4363, 'technosphere', 2), (4363, 'technosphere', 0.5), (1034, 'biosphere', 26.6)], 4363: [(4363, 'technosphere', 0.01), (4364, 'technosphere', 237)]}


In [14]:
import sys
sys.path.append("../")
from bw_scenarios.scenario import Scenario

test_scenario = Scenario("test")
test_scenario.exchanges = exc_dict


In [15]:
test_scenario.exchanges

{4365: [(4363, 'technosphere', 2),
  (4363, 'technosphere', 0.5),
  (1034, 'biosphere', 26.6)],
 4363: [(4363, 'technosphere', 0.01), (4364, 'technosphere', 237)]}

In [16]:
test_scenario.add_datapackage()

In [17]:
test_scenario.dp_static.data

[array([(4363, 4365), (4363, 4365), (4363, 4363), (4364, 4363)],
       dtype=[('row', '<i8'), ('col', '<i8')]),
 array([2.00e+00, 5.00e-01, 1.00e-02, 2.37e+02]),
 array([ True,  True,  True,  True]),
 array([(1034, 4365)], dtype=[('row', '<i8'), ('col', '<i8')]),
 array([26.6])]

In [18]:
bike = bd.get_node(name="production of bike")
method = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')

In [19]:
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {bike: 1},
    method=method
)
data_objs


In [20]:
data_objs[0].data

[array([], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([], dtype=float32),
 array([], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([], dtype=float32),
 array([], dtype=[('row', '<i4'), ('col', '<i4')]),
 array([], dtype=float32)]

In [21]:
data_objs.append(test_scenario.dp_static)
data_objs


In [22]:
my_lca = bc.LCA({bike.id:1}, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()
my_lca.score

26.6

In [ ]:
#TODO turn the cell above into a for-loop (quick fix instead of using MultiLCA)

In [27]:
data_objs[1].data

[array([(4363, 184), (4364, 175), (4365, 117)],
       dtype=[('row', '<i8'), ('col', '<i8')]),
 array([1., 1., 1.], dtype=float32),
 array([(1034, 4365)], dtype=[('row', '<i8'), ('col', '<i8')]),
 array([26.6], dtype=float32),
 array([(4363, 4363), (4363, 4363), (4363, 4365), (4363, 4365),
        (4364, 4363), (4364, 4364), (4365, 4365)],
       dtype=[('row', '<i8'), ('col', '<i8')]),
 array([1.00e-02, 1.00e+00, 5.00e-01, 2.00e+00, 2.37e+02, 1.00e+00,
        1.00e+00], dtype=float32),
 array([ True, False,  True,  True,  True, False, False])]